# **EQODEC: A Carbon-Aware Deep Learning Framework for Sustainable Video Compression**

---

This week marked the transition from data preparation to the full training and evaluation phase of EQODEC, our carbon-aware deep learning framework for sustainable video compression. Building on previously completed data collection and preprocessing workflows where Vimeo-90K septuplets and UVG videos were properly indexed and organized, the focus shifted toward finalizing the training pipeline, integrating the carbon-aware components, and optimizing the computational efficiency of the entire system.

The week's efforts concentrated on three major fronts:

1. Stabilizing and enhancing the EQODEC autoencoder training loop
2. Implementing high-fidelity evaluation methods, including EES computation
3. Improving runtime efficiency using AMP and ffmpeg streaming optimizations

These advancements move the project closer to fulfilling EQODEC's vision of balancing reconstruction quality, compression efficiency, and environmental impact as outlined in the Week-1 background document.

### **Model Training Pipeline Improvements**

**Final Integration of the ConvLSTM Autoencoder**

The ConvLSTM-based autoencoder which learns both spatial and temporal redundancies from the Vimeo-90K septuplets was fully integrated into the training loop. The model now performs:

- Frame-level spatial encoding
- Temporal modeling via ConvLSTM
- Latent quantization via a custom rounding module
- Two-stage decoding back to RGB reconstruction

This pipeline was successfully tested end-to-end using the preprocessed 7-frame sequences prepared last week.

**Clean Separation of Reconstruction and Carbon Losses**

A major architectural clean-up was completed:

- The **Reconstruction + BPP Loss** now contains no carbon-related terms.
- The **CarbonModule** is fully standalone, responsible only for computing a latent-level carbon proxy and managing CodeCarbon trackers.

This separation improves experimental flexibility, enabling future testing of different carbon-aware weighting strategies without disturbing core reconstruction logic.

**Incorporation of Carbon-Aware Loss During Training**

The training loop now computes the total loss as:

*Loss = MSE Reconstruction Loss + λ₍c₎ · Carbon Proxy*

Where:

- λ₍c₎ is currently set to 0.05.
- Carbon proxy = mean absolute latent magnitude × carbon intensity.

This aligns directly with Objective #2 in the Week-1 project background:
*Develop an end-to-end model with an energy-aware loss to encourage eco-friendly compression.*

---

### **Performance Optimization and Runtime Stability**

**Automatic Mixed Precision (AMP)**

To reduce GPU time and energy use, PyTorch AMP was integrated throughout:

- Training now uses `autocast()` and `GradScaler`.
- Inference phases inside the UVG evaluation also leverage AMP.

The result is:

- Lower memory consumption
- Faster step times
- Reduced energy usage (aligning with EQODEC’s sustainability goals)

**Improved FFmpeg-Based Streaming I/O**

The EES and evaluation pipeline required robust streaming of 1080p frames from UVG videos. This week introduced:

- New streaming reader using ffmpeg rawvideo pipe
- Stderr flushing logic to prevent ffmpeg stalls
- Chunk-based inference to avoid GPU OOM during resolution mismatch
- Real-time frame-level progress logging

These fixes resolved earlier issues where evaluation appeared "stuck" due to silent ffmpeg buffering.

---

### **Training Progress and Checkpointing**

**Full Training Runs**

The training loop was executed with:

- 2 epochs (test configuration)
- Adam optimizer (LR = 1e-4)
- Batch size 4 (Vimeo septuplets)

Both **training and validation PSNR and loss metrics** are now consistently logged per epoch.

When validation loss improved, the model automatically saved:

```
eqodec_best_epochX.pth
```

This ensures traceability and reproducibility of experiment results.

**Baseline Model Training**

The training loop was executed with:
- 2 epochs (test configuration)
- Adam optimizer (LR = 1e-4)
- Batch size 4 (Vimeo septuplets)

To prepare for comparative evaluation, a **baseline MSE-only autoencoder** was also trained:

- No carbon proxy
- Same architecture and hyperparameters

This enables direct comparison of:

- Reconstruction quality
- Bitrate performance
- Carbon footprint
- Final EES scores

as required in Objectives #2 and #3 of the EQODEC plan.

---

### **EES Computation and UVG Benchmark**

**Energy Efficiency Score (EES)**

A major achievement this week was the **successful deployment of the EES evaluator**, which combines:

- Baseline H.264 encoding
- Model-based reconstruction encoding
- CodeCarbon-tracked emissions
- Time-proportional attribution of energy cost
- Storage size comparison

EES expresses **kgCO₂ saved per GB of compressed video**, fulfilling the project's requirement to produce a sustainability-oriented metric.

**UVG Benchmark (PSNR)**

A refined evaluation loop now:

- Streams UVG 1080p videos
- Runs chunk-based inference
- Computes PSNR over full sequences

This high-resolution benchmark is directly connected to the data collection design in Week-1, where UVG was selected for its realism relative to traditional codecs' limits.

---

### **Summary**

This week's progress focused on fully stabilizing and optimizing the EQODEC model training pipeline, enabling end-to-end carbon-aware video compression using the Vimeo-90K and UVG datasets. The ConvLSTM autoencoder was successfully integrated with a clean reconstruction–carbon loss formulation, while a standalone CarbonModule ensured flexible computation of the sustainability objective introduced in the project background. Major performance improvements were achieved through Automatic Mixed Precision (AMP), ffmpeg-based streaming optimizations, and chunked 1080p inference, resulting in faster, more energy-efficient training and evaluation. Full training of both the EQODEC model and the MSE-only baseline was completed, with automatic checkpointing and consistent PSNR tracking. Additionally, the Energy Efficiency Score (EES) evaluator and UVG benchmark were fully operational, enabling realistic high-resolution testing and carbon-footprint assessment. Overall, this week established a stable, efficient, and carbon-aware training and evaluation workflow, marking a significant advancement toward EQODEC's sustainability-centered objectives.